#### Dataset 출처: 파이썬 라이브러리 스파르타 노션 교재

- 기본 WebBaseLoader로 긁어온다면 API token이 없기 때문에 불가했기 떄문
- 교재 url은 .env파일에 저장됨


In [29]:
import os
from dotenv import load_dotenv

load_dotenv()
import os

urls = []

# 1부터 17까지의 환경 변수 이름을 동적으로 생성하여 값을 가져오기
for i in range(1, 18):
    url = os.getenv(f"PYTHON_LIBRARY_URL_{i}")  # 환경 변수 이름 생성
    if url:  # 값이 None이 아닌 경우만 추가
        urls.append(url)

# # URL 목록 출력
# print(urls)


headers = {
    "User-Agent": "yMozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"  
}

### 데이터 전처리 및 동적 크롤링 (윤서님 코드 참고)

In [8]:
import re

def extract_page_id(notion_url: str) -> str:
    # Notion URL에서 ID 추출 (UUID)
    match = re.search(r'([a-f0-9]{32})', notion_url)
    if match:
        page_id = match.group(1)
        # UUID 포맷으로 변환
        return f"{page_id[:8]}-{page_id[8:12]}-{page_id[12:16]}-{page_id[16:20]}-{page_id[20:]}"
    else:
        raise ValueError("유효한 Notion 페이지 URL이 아닙니다.")

In [4]:
import re
import time
import nltk
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ssl

In [5]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [6]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/suyeon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /Users/suyeon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/suyeon/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#### 전처리 부분 (코드 블럭과 설명을 구별하지 않고 받아옴)

In [7]:
def preprocess_text(text):
    # 1. HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)  # HTML 태그 제거

    # 2. 소문자 변환
    text = text.lower()

    # 3. 구두점 및 특수 문자 제거
    text = re.sub(r'[^\w\s]', '', text)

    # 4. 불용어 제거
    stop_words = set(stopwords.words('english'))  # 영어 불용어 목록
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]

    # 5. 전처리된 텍스트 반환
    return ' '.join(filtered_text)

### TEXT 파일 저장
- txt 파일로 저장하고자하는 이유는 단순 문자열이기 때문
- 저장 위치 설정

In [36]:
import os

root_path = "python_library_"

def save_to_txt(text: str, file_name: str):
    try:
        os.makedirs(root_path, exist_ok=True)

        # 파일 경로 설정
        raw_file_path = os.path.join(root_path, f"{file_name}.txt")

        # 텍스트 파일로 저장
        with open(raw_file_path, 'w', encoding='utf-8') as code_file:
            code_file.write(text)

        print(f"TXT 파일 저장 완료:\n- raw: {raw_file_path}")

    except Exception as e:
        print(f"TXT 저장 중 오류 발생: {e}")



### 정보를 긁어오고 저장하는 함수 (body에 있는 모든 내용을 긁어옴)

In [37]:
def fetch_page_content(urls):
    # Selenium 웹드라이버 설정
    # Selenium 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 브라우저 창을 표시하지 않음
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


    i = 0
    # 각 URL을 순차적으로 처리
    for url in urls:
        i += 1
        driver.get(url)
        time.sleep(5)  # 페이지 로딩 대기 (필요에 따라 조정)

        # 페이지 내용 추출 (body 태그 내의 텍스트)
        page_content = driver.find_element(By.TAG_NAME, "body").text
        # 텍스트 전처리 후 저장
        preprocessed_text = preprocess_text(page_content)
        save_to_txt(preprocessed_text, f"python_library-{str(i)}")

    # 브라우저 종료
    driver.quit()

#### 최종 실행 및 저장 부분

In [38]:
fetch_page_content(urls)

TXT 파일 저장 완료:
- raw: python_library_/python_library-1.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-2.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-3.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-4.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-5.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-6.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-7.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-8.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-9.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-10.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-11.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-12.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-13.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-14.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-15.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-16.txt
TXT 파일 저장 완료:
- raw: python_library_/python_library-17.txt
